In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNetCV, LogisticRegressionCV
from sklearn.metrics import accuracy_score

df_movies = pd.read_csv('data/movie_reviews_processed.csv')

X = df_movies.filter(regex='_sc$')
y = df_movies['rating_good']

In [32]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
X = heart_disease.data.features
y = heart_disease.data.targets

# metadata
# print(heart_disease.metadata)

# variable information
# print(heart_disease.variables)

df_heart = (
    X
    .assign(
        cp = pd.Categorical(X.cp).rename_categories(['typical', 'atypical', 'non-anginal', 'asymptomatic']),
        fbs = pd.Categorical(X.fbs).rename_categories(['<= 120 mg/dl', '> 120 mg/dl']),
        restecg = pd.Categorical(X.restecg).rename_categories(['normal', 'ST-T wave abnormality', 'left ventricular hypertrophy']),
        exang = pd.Categorical(X.exang).rename_categories(['no', 'yes']),
        slope = pd.Categorical(X.slope).rename_categories(['upsloping', 'flat', 'downsloping']),
        # ca = pd.Categorical(X.ca, categories = ['0', '1', '2', '3']),
        thal = pd.Categorical(X.thal).rename_categories(['normal', 'fixed defect', 'reversible defect']),
        heart_disease = pd.Categorical(y.num > 0).rename_categories(['no', 'yes'])
    )
    .rename(
        columns={
            'sex': 'male',
            'cp': 'chest_pain_type',
            'trestbps': 'resting_bp',
            'chol': 'cholesterol',
            'fbs': 'fasting_blood_sugar',
            'restecg': 'resting_ecg',
            'thalach': 'max_heart_rate',
            'exang': 'exercise_induced_angina',
            'oldpeak': 'st_depression',
            'ca': 'num_major_vessels',
            'thal': 'thalassemia',
        }
    )
    .assign(
        fasting_blood_sugar_num=lambda x: x.fasting_blood_sugar.cat.codes,
        resting_ecg_num=lambda x: x.resting_ecg.cat.codes,
        exercise_induced_angina_num=lambda x: x.exercise_induced_angina.cat.codes,
        heart_disease_num=lambda x: x.heart_disease.cat.codes,
    )
)

df_heart.head()

,age,male,chest_pain_type,resting_bp,cholesterol,fasting_blood_sugar,resting_ecg,max_heart_rate,exercise_induced_angina,st_depression,slope,num_major_vessels,thalassemia,heart_disease,fasting_blood_sugar_num,resting_ecg_num,exercise_induced_angina_num,heart_disease_num
0,63,1,typical,145,233,> 120 mg/dl,left ventricular hypertrophy,150,no,2.3,downsloping,0.0,fixed defect,no,1,2,0,0
1,67,1,asymptomatic,160,286,<= 120 mg/dl,left ventricular hypertrophy,108,yes,1.5,flat,3.0,normal,yes,0,2,1,1
2,67,1,asymptomatic,120,229,<= 120 mg/dl,left ventricular hypertrophy,129,yes,2.6,flat,2.0,reversible defect,yes,0,2,1,1
3,37,1,non-anginal,130,250,<= 120 mg/dl,normal,187,no,3.5,downsloping,0.0,normal,no,0,0,0,0
4,41,0,atypical,130,204,<= 120 mg/dl,left ventricular hypertrophy,172,no,1.4,upsloping,0.0,normal,no,0,2,0,0


In [31]:
pd.Categorical(y.num)#.rename_categories(['typical', 'atypical', 'non-anginal', 'asymptomatic'])

[0, 2, 1, 0, 0, ..., 1, 2, 3, 1, 0]
Length: 303
Categories (5, int64): [0, 1, 2, 3, 4]

In [2]:
# Fit model

model = LogisticRegressionCV(
  penalty   = 'elasticnet', 
  l1_ratios = np.arange(.0, 1, .05), 
  cv     = 5, 
  solver = 'saga', 
  random_state = 123,
  verbose = False
)

model.fit(X, y)

# Predict

y_pred = model.predict(X)

# Evaluate

print(
  'Training accuracy: ', accuracy_score(y, y_pred),
  'Baseline Prevalence: ', np.mean(y)
)

Training accuracy:  0.68 Baseline Prevalence:  0.563
